In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


# tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

2.1.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [6]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [16]:
print(training_sentences[:2])
print(training_labels[:2])

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."', "b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot develo

In [11]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [17]:
padded[:2]

array([[   0,    0,   59,   12,   14,   35,  439,  400,   18,  174,   29,
           1,    9,   33, 1378, 3401,   42,  496,    1,  197,   25,   88,
         156,   19,   12,  211,  340,   29,   70,  248,  213,    9,  486,
          62,   70,   88,  116,   99,   24, 5740,   12, 3317,  657,  777,
          12,   18,    7,   35,  406, 8228,  178, 2477,  426,    2,   92,
        1253,  140,   72,  149,   55,    2,    1, 7525,   72,  229,   70,
        2962,   16,    1, 2880,    1,    1, 1506, 4998,    3,   40, 3947,
         119, 1608,   17, 3401,   14,  163,   19,    4, 1253,  927, 7986,
           9,    4,   18,   13,   14, 4200,    5,  102,  148, 1237,   11,
         240,  692,   13,   44,   25,  101,   39,   12, 7232,    1,   39,
        1378,    1,   52,  409,   11,   99, 1214,  874,  145,   10],
       [   0,    0,    0,    0,    0,    0,    0,  256,   28,   78,  585,
           6,  815, 2383,  317,  109,   19,   12,    7,  643,  696,    6,
           4, 2249,    5,  183,  599,   68,

In [23]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[2]))
print(training_sentences[2])

<OOV> photographs the <OOV> rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of dawson city <OOV> to <OOV> themselves a <OOV> yes a <OOV> and to <OOV> the law themselves then <OOV> battling it out on the streets for control of the town br br nothing even remotely resembling that happened on the canadian side of the border during the <OOV> gold rush mr mann and company appear to have mistaken dawson city for <OOV> the canadian north for the american wild west br br canadian viewers be prepared for a <OOV> madness type of enjoyable
b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, th

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [25]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 103us/sample - loss: 0.5745 - accuracy: 0.7091 - val_loss: 0.4496 - val_accuracy: 0.8390
Epoch 2/10
25000/25000 [==============================] - 2s 84us/sample - loss: 0.3479 - accuracy: 0.8912 - val_loss: 0.4149 - val_accuracy: 0.8356
Epoch 3/10
25000/25000 [==============================] - 2s 84us/sample - loss: 0.1942 - accuracy: 0.9570 - val_loss: 0.4619 - val_accuracy: 0.8243
Epoch 4/10
25000/25000 [==============================] - 2s 84us/sample - loss: 0.1159 - accuracy: 0.9798 - val_loss: 0.5356 - val_accuracy: 0.8248
Epoch 5/10
25000/25000 [==============================] - 2s 84us/sample - loss: 0.0828 - accuracy: 0.9862 - val_loss: 0.5254 - val_accuracy: 0.8230
Epoch 6/10
25000/25000 [==============================] - 2s 84us/sample - loss: 0.0684 - accuracy: 0.9876 - val_loss: 0.5665 - val_accuracy: 0.8260
Epoch 7/10
25000/25000 [==============================]

In [27]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [28]:
weights[:2]

array([[ 0.02318902,  0.04706934,  0.02413073, -0.03792838, -0.00716944,
         0.01683171, -0.021222  ,  0.01191443,  0.00862822, -0.00810603,
         0.02528671, -0.01878581, -0.0036978 , -0.00943623, -0.00945972,
         0.00112321],
       [ 0.03832161,  0.05474515,  0.06734503,  0.00352726,  0.02605384,
        -0.04496339,  0.0140282 , -0.02251247, -0.02217355,  0.03069763,
        -0.021356  ,  0.00073224, -0.00891718,  0.00604913, -0.01743244,
         0.00021792]], dtype=float32)

In [29]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [30]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [32]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]
